# ACT2023 Tutorial Sessions - exercises part 2 
init draft: 05 Aug 2023
last update: 05 Aug 2023

Ref: \
https://act2023tutorials.netlify.app/algebraicjulia/exercises \
https://act2023tutorials.netlify.app/algebraicjulia/solutions

## Exercise (Growing rabbits)

In [1]:
using AlgebraicDynamics

ṙ(r,p,t) = p.α * r
rabbit_growth = ContinuousResourceSharer{Float64}(1, ṙ)

ContinuousResourceSharer(ℝ^1 → ℝ^1) with 1 exposed port

In [ ]:
# import Pkg; Pkg.add("LabelledArrays")
# import Pkg; Pkg.add("DifferentialEquations")

In [4]:
using LabelledArrays
using DifferentialEquations, Plots

u0 = [1.0]
params = LVector(α=0.3)
tspan = (0.0, 10.0)

prob = ODEProblem(rabbit_growth, u0, tspan, params)
sol = solve(prob, Tsit5())

plot(sol, title="Exponential growth of rabbit population",
     legend=false, xlabel="time", ylabel="population size")

LoadError: ArgumentError: Package DifferentialEquations not found in current path.
- Run `import Pkg; Pkg.add("DifferentialEquations")` to install the DifferentialEquations package.

In [5]:
import Pkg; Pkg.add("DifferentialEquations")

   Resolving package versions...
   Installed StatsFuns ───────────────── v1.3.0
   Installed TreeViews ───────────────── v0.3.0
   Installed Calculus ────────────────── v0.5.1
   Installed PDMats ──────────────────── v0.11.17
   Installed EnumX ───────────────────── v1.0.4
   Installed Polyester ───────────────── v0.7.5
   Installed HypergeometricFunctions ─── v0.3.23
   Installed NonlinearSolve ──────────── v1.9.0
   Installed DifferentialEquations ───── v7.8.0
   Installed Sundials_jll ────────────── v5.2.1+0
   Installed CEnum ───────────────────── v0.4.2
   Installed ZygoteRules ─────────────── v0.2.3
   Installed FunctionWrappers ────────── v1.1.3
   Installed TriangularSolve ─────────── v0.1.19
   Installed Distances ───────────────── v0.10.9
   Installed BoundaryValueDiffEq ─────── v2.11.0
   Installed RandomNumbers ───────────── v1.5.3
   Installed SteadyStateDiffEq ───────── v1.16.0
   Installed Tricks ──────────────────── v0.1.7
   Installed FiniteDiff ──────────────── v2.21

## Exercise (UWDs as data structure)

In [ ]:
uwd = @relation (x,y,z) where (w,x,y,z) begin
  R(x,w)
  S(y,w)
  T(z,w)
end

to_graphviz(uwd, box_labels=:name, junction_labels=:variable)

In [ ]:
d = RelationDiagram(3) # Create diagram with 3 outer ports
add_junctions!(d, 4, variable=[:w,:x,:y,:z]) # Add four junctions
set_junction!(d, 1:3, 2:4, outer=true) # Set junctions of outer ports

for name in [:R, :S, :T]
  add_box!(d, 2, name=name) # Add box with two ports
end
for (i, box) in enumerate(boxes(d))
  set_junction!(d, (box, 1), i+1)
  set_junction!(d, (box, 2), 1)
end

@test d == uwd

## Exercise (Pendulum)

In [6]:
du(u,p,t) = [u[2], -p.g/p.ℓ * sin(u[1])]

pendulum = ContinuousResourceSharer{Float64}(2, du)

u0 = [π/4, 0]
params = LVector(g=9.8, ℓ=1)
tspan = (0.0, 10.0)

prob = ODEProblem(pendulum, u0, tspan, params)
sol = solve(prob, Tsit5())
plot(sol, title="Simple pendulum", xlabel="time", label=["θ" "ω"])

LoadError: UndefVarError: `ODEProblem` not defined

## Exercise (Graph traversal)

In [ ]:
using Catlab
using DataStructures: Stack

""" Depth-first search in graph `g` starting at vertex `v`

The function `f` is called at every vertex in the search path.
"""
function dfs(f::Function, g::ACSet, v::Int)
  seen = Set{Int}()
  next = Stack{Int}()
  push!(next, v)
  while !isempty(next)
    v = pop!(next)
    v ∈ seen && continue
    f(v)
    push!(seen, v)
    # Add all outgoing edges to the stack.
    for e in incident(g, v, :src)
      # Syntactic sugar: `g[e, :tgt] == subpart(g, e, :tgt)``
      push!(next, g[e, :tgt])
    end
  end
end

In [ ]:
g = cycle_graph(Graph, 6)
dfs(println, g, 3)

## Exercise (Union-find)

In [ ]:
using Catlab
using DataStructures: IntDisjointSets, union!, find_root

""" Find connected components of a graph.

Returns a dictionary from the component roots to lists of vertices in each
component.
"""
function connected_components(g::HasGraph)
  sets = IntDisjointSets(nparts(g, :V))
  for e in parts(g, :E)
    union!(sets, g[e,:src], g[e,:tgt])
  end

  components = Dict{Int,Vector{Int}}()
  for v in parts(g, :V)
    # The first argument inserts an empty array
    # when v's connected component has not yet been visited.
    component = get!(() -> Int[], components, find_root(sets, v))
    push!(component, v)
  end
  components
end

In [ ]:
g = path_graph(Graph, 3) ⊕ cycle_graph(Graph, 4) ⊕ star_graph(Graph, 4)

values(connected_components(g))

## Exercise (Julia macros)

In [ ]:
macro make_graph(expr)
  make_graph(expr)
end

function make_graph(block::Expr)
  g = Graph()
  vnames = Dict{Symbol,Int}()
  block.head == :block || error("Input to `make_graph` must be a block")
  for arg in block.args
    if arg isa LineNumberNode
      continue
    elseif arg isa Symbol
      haskey(vnames, arg) && error("Vertex $arg already defined")
      vnames[arg] = add_vertex!(g)
    elseif arg isa Expr && arg.head == :(->)
      sname, rhs = arg.args
      tname = rhs.args[2]
      add_edge!(g, vnames[sname], vnames[tname])
    else
      error("Cannot parse statement $arg")
    end
  end
  g
end

In [ ]:
g = @make_graph begin
  a
  b
  c
  a -> b
  b -> c
end

to_graphviz(g)